### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

In [6]:
import oci
from langchain_community.document_loaders.llmsherpa import LLMSherpaFileLoader

In [5]:
!pip install langchain-community langchain-core

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 30.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 60.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 60.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 42.2 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [9]:
!pip install llmsherpa

In [83]:
llmsherpa_api_url = "http://149.130.214.34:5010/api/parseDocument?renderFormat=all"

#### Section wise chunking

In [86]:
def convert_to_text_sherpa(pdf_url: str):    

        loader = LLMSherpaFileLoader(
            file_path=pdf_url,
            new_indent_parser=True,
            apply_ocr=True,
            strategy="sections",
            llmsherpa_api_url=llmsherpa_api_url,
        )
        docs = loader.load()
        return docs

In [97]:
doc_to_text = convert_to_text_sherpa(r'TWB US Millennium Steel Inv # 54063.pdf')
print(doc_to_text)

[Document(metadata={'source': 'TWB US Millennium Steel Inv # 54063.pdf', 'section_number': 0, 'section_title': 'Odyssey'}, page_content='Odyssey'), Document(metadata={'source': 'TWB US Millennium Steel Inv # 54063.pdf', 'section_number': 1, 'section_title': 'Odyssey      Specialized      Logistics      LLC'}, page_content='Odyssey      Specialized      Logistics      LLC\nat:      PORTAGE,      IN'), Document(metadata={'source': 'TWB US Millennium Steel Inv # 54063.pdf', 'section_number': 2, 'section_title': 'TWB      COMPANY      LLC       C/O      TOYOTA'}, page_content='TWB      COMPANY      LLC       C/O      TOYOTA'), Document(metadata={'source': 'TWB US Millennium Steel Inv # 54063.pdf', 'section_number': 3, 'section_title': 'MI      48162'}, page_content='MI      48162\nL{-y      aod      Ink      indus      paral      ce      t4/han      and      ret   \n \n7      ie      apse      ae  \n \n  Bieieaatet      ree      rey      Lg\nShipper:      CLIFFS      STEEL      INC\nFax   

In [98]:
document_sections = [
    {"section_title": doc.metadata["section_title"], "content": doc.page_content}
    for doc in doc_to_text  # Replace 'doc_to_text' with the actual list variable holding your Document objects
]

In [99]:
document_sections

[{'section_title': 'Odyssey', 'content': 'Odyssey'},
 {'section_title': 'Odyssey      Specialized      Logistics      LLC',
  'content': 'Odyssey      Specialized      Logistics      LLC\nat:      PORTAGE,      IN'},
 {'section_title': 'TWB      COMPANY      LLC       C/O      TOYOTA',
  'content': 'TWB      COMPANY      LLC       C/O      TOYOTA'},
 {'section_title': 'MI      48162',
  'content': 'MI      48162\nL{-y      aod      Ink      indus      paral      ce      t4/han      and      ret   \n \n7      ie      apse      ae  \n \n  Bieieaatet      ree      rey      Lg\nShipper:      CLIFFS      STEEL      INC\nFax      Person:\nFax      Number:\nPOs:      BH51341004A\nEnd      Use:\nCustPo#:      042423\nMill      Order#:      ~BH51341004A\nParts:      58111/2-A1\nDuns#:      789153079\nGrade:      9312\nORDER      INFO\nDIMENSIONS      (Gauge/Width)\n-0472x54      3306\nGALVANIZED 1.1995x1380      .67mm\n68705080\nCust.      Rel: Deliver      Date:      6/14/2024 Time:\nFreight  

#### Project details document

In [40]:
import oci
import json
import re

# Configuration and Client Setup
compartment_id = "ocid1.compartment.oc1..aaaaaaaaretksgipt3jgwfpzgh4ijyw54uynyfviaxs5li4wtl744fj4fi3q"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('config', CONFIG_PROFILE)
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10,240))

# Function to extract entities for each section
def extract_entities_from_section(section_content):
    chat_detail = oci.generative_ai_inference.models.ChatDetails()
    chat_request = oci.generative_ai_inference.models.CohereChatRequest()
    chat_request.message = f"Extract only the project manager’s email or name from the section below and present it in JSON format. Do not include any additional information.\n\n{section_content}"
    chat_request.max_tokens = 600
    chat_request.temperature = 0
    chat_request.frequency_penalty = 0
    chat_request.top_p = 0.75
    chat_request.top_k = 0
    chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyawk6mgunzodenakhkuwxanvt6wo3jcpf72ln52dymk4wq")
    chat_detail.chat_request = chat_request
    chat_detail.compartment_id = compartment_id
    chat_response = generative_ai_inference_client.chat(chat_detail)
    return chat_response

results = []
for section in document_sections:
    extracted_data = extract_entities_from_section(section['content'])
    extracted_text = extracted_data.data.chat_response.text
    print('***********extracted_text_step1**********',extracted_text)
    # Use regex to extract JSON content within backticks if necessary
    json_match = re.search(r"```json(.*?)```", extracted_text, re.DOTALL)
    # print('***********json_match**********',json_match)
    if json_match:
        extracted_text = json_match.group(1).strip()
    else:
        extracted_text = extracted_text.strip("```").strip("json\n")
    #Attempt to load JSON, with additional handling if necessary
    try:
        results.append({
            "section_title": section["section_title"],
            "extracted_data": json.loads(extracted_text)
        })
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for section '{section['section_title']}': {e}")
        print("Raw extracted text:", extracted_text)
        # Append an error note or skip this section if JSON fails to parse
        results.append({
            "section_title": section["section_title"],
            "extracted_data": "Error: Could not parse JSON"
        })

print("**************************Extracted Education Details**************************")
print(json.dumps(results, indent=2))

***********extracted_text_step1********** ```json
[
    "laurence.cellier@oracle.com",
    "vishnukanth.gollapotu@oracle.com"
]
```
***********extracted_text_step1********** ```json
[
    {"name":"Brian Jones"}
]
```
***********extracted_text_step1********** ```json
[ 
    {"name":"Egbert Daskowski"}
]
```
***********extracted_text_step1********** ```json
[ 
    {"name":"Henry Gibson", "email":"hgibson0@ucoz.com"},
    {"name":"Florence Thompson", "email":"fthompson1@webnode.com"},
    {"email":"jcollingwood2@hatenablog.com"},
    {"name":"Cindy Hayward", "email":"chayward3@skyrock.com"},
    {"email":"mstokes4@webs.com"},
    {"name":"Michael Clark", "email":"mclark5@devhub.com"}
]
```
***********extracted_text_step1********** ```json
[
    "TM"
]
```
***********extracted_text_step1********** ```json
[ 
    {"name":"Henry Jenkins", "email":"jenkins.henry@email.com"},
    {"name":"Jane Smith", "email":"smith.jane@email.com"}
]
```
***********extracted_text_step1********** ```json
[
   

In [46]:
import oci
import json
import re

# Configuration and Client Setup
compartment_id = "ocid1.compartment.oc1..aaaaaaaaretksgipt3jgwfpzgh4ijyw54uynyfviaxs5li4wtl744fj4fi3q"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('config', CONFIG_PROFILE)
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10, 240)
)

# Function to extract entities from the entire document content
def extract_entities_from_document(full_content):
    chat_detail = oci.generative_ai_inference.models.ChatDetails()
    chat_request = oci.generative_ai_inference.models.CohereChatRequest()
    chat_request.message = (
        f"Extract the project number, project name, customer name and all the project manager’s name and email from the content below and present it as structured JSON objects. \n\n{full_content}"
    )
    chat_request.max_tokens = 600
    chat_request.temperature = 0
    chat_request.frequency_penalty = 0
    chat_request.top_p = 0.75
    chat_request.top_k = 0
    chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(
        model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyawk6mgunzodenakhkuwxanvt6wo3jcpf72ln52dymk4wq"
    )
    chat_detail.chat_request = chat_request
    chat_detail.compartment_id = compartment_id
    chat_response = generative_ai_inference_client.chat(chat_detail)
    return chat_response

# Concatenate all section content into one string
full_content = "\n".join([section['content'] for section in document_sections])

# Extract entities from the full content at once
extracted_data = extract_entities_from_document(full_content)
extracted_text = extracted_data.data.chat_response.text

# Use regex to extract JSON content within backticks if necessary
json_match = re.search(r"```json(.*?)```", extracted_text, re.DOTALL)
if json_match:
    extracted_text = json_match.group(1).strip()
else:
    extracted_text = extracted_text.strip("```").strip("json\n")

# Attempt to load JSON, with additional handling if necessary
results = []
try:
    results = json.loads(extracted_text)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON from document: {e}")
    print("Raw extracted text:", extracted_text)
    results = "Error: Could not parse JSON"
print("**************************Extracted Project Information**************************")
print(json.dumps(results, indent=2))

**************************Extracted Project Manager Information**************************
[
  {
    "project_number": 400316712,
    "project_name": "A6MNJK_TM_AM_FR_EGIS SA_Integration_ERP with OIC",
    "customer_name": "EGIS SA",
    "project_manager": {
      "name": "Laurence Cellier",
      "email": "laurence.cellier@oracle.com"
    },
    "secondary_project_manager": {
      "name": "Vishnukanth Gollapotu",
      "email": "vishnukanth.gollapotu@oracle.com"
    }
  }
]


#### Readable invoice single prompt approach

In [100]:
import oci
import json
import re

# Configuration and Client Setup
compartment_id = "ocid1.compartment.oc1..aaaaaaaaretksgipt3jgwfpzgh4ijyw54uynyfviaxs5li4wtl744fj4fi3q"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('config', CONFIG_PROFILE)
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10, 240)
)

# Function to extract entities from the entire document content
def extract_entities_from_document(full_content):
    chat_detail = oci.generative_ai_inference.models.ChatDetails()
    chat_request = oci.generative_ai_inference.models.CohereChatRequest()
    chat_request.message = (
        f"Extract all invoice details, customer and vendor information, itemized charges, tax and payment details, and any additional instructions from the following invoice text. Present each section in JSON format with clear labels for each piece of information. \n\n{full_content}"
    )
    chat_request.max_tokens = 600
    chat_request.temperature = 0
    chat_request.frequency_penalty = 0
    chat_request.top_p = 0.75
    chat_request.top_k = 0
    chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(
        model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyawk6mgunzodenakhkuwxanvt6wo3jcpf72ln52dymk4wq"
    )
    chat_detail.chat_request = chat_request
    chat_detail.compartment_id = compartment_id
    chat_response = generative_ai_inference_client.chat(chat_detail)
    return chat_response


full_content = "\n".join([section['content'] for section in document_sections])
extracted_data = extract_entities_from_document(full_content)
extracted_text = extracted_data.data.chat_response.text

json_match = re.search(r"```json(.*?)```", extracted_text, re.DOTALL)
if json_match:
    extracted_text = json_match.group(1).strip()
else:
    extracted_text = extracted_text.strip("```").strip("json\n")

# Attempt to load JSON, with additional handling if necessary
results = []
try:
    results = json.loads(extracted_text)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON from document: {e}")
    print("Raw extracted text:", extracted_text)
    results = "Error: Could not parse JSON"
print("**************************Extracted Project Information**************************")
print(json.dumps(results, indent=2))

**************************Extracted Project Information**************************
[
  {
    "invoice_details": {
      "invoice_number": 1106786,
      "date": "6/14/2024"
    },
    "customer": {
      "name": "Cliffs Steel Inc",
      "fax_person": "",
      "fax_number": "",
      "po_number": "BH51341004A",
      "end_use": "",
      "cust_po": "042423",
      "mill_order": "~BH51341004A",
      "parts": "58111/2-A1",
      "duns_number": "789153079",
      "grade": "9312"
    },
    "vendor": {
      "name": "Odyssey Specialized Logistics LLC",
      "address": "PORTAGE, IN"
    },
    "itemized_charges": [
      {
        "item_description": "GALVANIZED 1.1995x1380 .67mm",
        "dimensions": "-0472x54 3306",
        "quantity": 1,
        "unit_price": "",
        "total_price": ""
      },
      {
        "item_description": "COIL Q LOAD EYE LEWSE",
        "dimensions": "",
        "quantity": 1,
        "unit_price": "",
        "total_price": ""
      }
    ],
    "tax_det

#### Readable Invoice multi-prompt approach

In [59]:
import oci
import json
import re

# Configuration and Client Setup
compartment_id = "ocid1.compartment.oc1..aaaaaaaaretksgipt3jgwfpzgh4ijyw54uynyfviaxs5li4wtl744fj4fi3q"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('config', CONFIG_PROFILE)
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10, 240)
)

# Function to extract entities from the entire document content based on multiple prompts
def extract_entities_from_document(full_content):
    chat_detail = oci.generative_ai_inference.models.ChatDetails()
    chat_request = oci.generative_ai_inference.models.CohereChatRequest()
    
    # Define prompts for different entities
    prompts = {
        "invoice_details": "Extract the invoice number and date from the following text and present it in JSON format.",
        "customer_info": "Extract customer details including name, registered office, GST, and CIN in JSON format.",
        "vendor_info": "Extract vendor details including name, address, state code, and GST number in JSON format.",
        "itemized_charges": "Extract item details including item code, quantity, total value, and rates (IGST, SGST, CGST) in JSON format.",
        "tax_details": "Extract tax details including cess value, total invoice value, advance received, and total in words in JSON format.",
        "payment_details": "Extract payment terms, amount in words, and total IGST in JSON format.",
        "additional_info": "Extract IRN number, acknowledgement number, and eway bill number in JSON format."
    }

    results = {}
    for key, prompt in prompts.items():
        chat_request.message = f"{prompt}\n\n{full_content}"
        chat_request.max_tokens = 600
        chat_request.temperature = 0
        chat_request.frequency_penalty = 0
        chat_request.top_p = 0.75
        chat_request.top_k = 0
        chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(
            model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyawk6mgunzodenakhkuwxanvt6wo3jcpf72ln52dymk4wq"
        )
        chat_detail.chat_request = chat_request
        chat_detail.compartment_id = compartment_id
        chat_response = generative_ai_inference_client.chat(chat_detail)
        extracted_text = chat_response.data.chat_response.text
        json_match = re.search(r"```json(.*?)```", extracted_text, re.DOTALL)
        if json_match:
            extracted_text = json_match.group(1).strip()
        else:
            extracted_text = extracted_text.strip("```").strip("json\n")
        try:
            results[key] = json.loads(extracted_text)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for {key}: {e}")
            print("Raw extracted text:", extracted_text)
            results[key] = "Error: Could not parse JSON"

    return results
full_content = "\n".join([section['content'] for section in document_sections])
results = extract_entities_from_document(full_content)
print("**************************Extracted Project Information**************************")
print(json.dumps(results, indent=2))

**************************Extracted Project Information**************************
{
  "invoice_details": [
    {
      "invoice_number": "5MT24251603 M337",
      "date": "01/05/2024"
    }
  ],
  "customer_info": {
    "customer_details": {
      "name": "MOTHERSON AIR TRAVEL AGENCIES LTD.",
      "registered_office": "86, Sukhdev Vihar, Mathura Road, New Delhi - 11002",
      "GST": "07AAACM1993R1ZT",
      "CIN": "U74899DL1994PLC058171"
    }
  },
  "vendor_info": [
    {
      "vendor_name": "MOTHERSON AIR TRAVEL AGENCIES LTD.",
      "address": "86, Sukhdev Vihar, Mathura Road,, New Delhi - 11002",
      "state_code": "DL",
      "GST_number": "AAACM1993RST001"
    }
  ],
  "itemized_charges": [
    {
      "item_code": "AAACM1993RST001",
      "quantity": 1,
      "total_value": "4213.00",
      "rates": {
        "IGST": "643.00",
        "SGST": "0.00",
        "CGST": "0.00"
      }
    },
    {
      "item_code": "998559",
      "quantity": 2,
      "total_value": "7140.00",


#### Image invoice

In [67]:
import oci
import json
import re

# Configuration and Client Setup
compartment_id = "ocid1.compartment.oc1..aaaaaaaaretksgipt3jgwfpzgh4ijyw54uynyfviaxs5li4wtl744fj4fi3q"
CONFIG_PROFILE = "DEFAULT"
config = oci.config.from_file('config', CONFIG_PROFILE)
endpoint = "https://inference.generativeai.us-chicago-1.oci.oraclecloud.com"
generative_ai_inference_client = oci.generative_ai_inference.GenerativeAiInferenceClient(
    config=config, service_endpoint=endpoint, retry_strategy=oci.retry.NoneRetryStrategy(), timeout=(10, 240)
)

# Function to extract entities from the entire document content based on multiple prompts
def extract_entities_from_document(full_content):
    chat_detail = oci.generative_ai_inference.models.ChatDetails()
    chat_request = oci.generative_ai_inference.models.CohereChatRequest()
    prompts = {
        "invoice_details": "Extract the invoice number and date from the following text and present it in JSON format.",
        "customer_info": "Extract customer details including name, registered office, GST, and CIN in JSON format.",
        "vendor_info": "Extract vendor details including name, address, state code, and GST number in JSON format.",
        "itemized_charges": "Extract item details including item code, quantity, total value, and rates (IGST, SGST, CGST) in JSON format.",
        "tax_details": "Extract tax details including cess value, total invoice value, advance received, and total in words in JSON format.",
        "payment_details": "Extract payment terms, amount in words, and total IGST in JSON format.",
        "additional_info": "Extract IRN number, acknowledgement number, and eway bill number in JSON format."
    }

    results = {}
    for key, prompt in prompts.items():
        chat_request.message = f"{prompt}\n\n{full_content}"
        chat_request.max_tokens = 600
        chat_request.temperature = 0
        chat_request.frequency_penalty = 0
        chat_request.top_p = 0.75
        chat_request.top_k = 0
        chat_detail.serving_mode = oci.generative_ai_inference.models.OnDemandServingMode(
            model_id="ocid1.generativeaimodel.oc1.us-chicago-1.amaaaaaask7dceyawk6mgunzodenakhkuwxanvt6wo3jcpf72ln52dymk4wq"
        )
        chat_detail.chat_request = chat_request
        chat_detail.compartment_id = compartment_id
        chat_response = generative_ai_inference_client.chat(chat_detail)
        extracted_text = chat_response.data.chat_response.text
        json_match = re.search(r"```json(.*?)```", extracted_text, re.DOTALL)
        if json_match:
            extracted_text = json_match.group(1).strip()
        else:
            extracted_text = extracted_text.strip("```").strip("json\n")
        try:
            results[key] = json.loads(extracted_text)
        except json.JSONDecodeError as e:
            print(f"Error decoding JSON for {key}: {e}")
            print("Raw extracted text:", extracted_text)
            results[key] = "Error: Could not parse JSON"

    return results


full_content = "\n".join([section['content'] for section in document_sections])
results = extract_entities_from_document(full_content)

print("**************************Extracted Project Information**************************")
print(json.dumps(results, indent=2))

**************************Extracted Project Information**************************
{
  "invoice_details": {
    "invoice_number": "ATT/24-25/028",
    "invoice_date": "31-May-24"
  },
  "customer_info": [
    {
      "name": "Awana Tour & Travels",
      "registered_office": "Street No. 02, Gijhore, Near Kanchan Junga, Appartment, Sector-53, Noida. 201301 (UP)",
      "mobile_numbers": [
        "+91-9811100310",
        "+9999374459"
      ],
      "email": "awanatravels@yahoo.com",
      "website": "www.awanatravels.co.in",
      "GST": "OSAANCM5330P12T",
      "CIN": "O9ABMPA0354N12ZJ"
    },
    {
      "name": "Motherson Sumi Wiring India Ltd",
      "registered_office": "C-14, Sector-01, Noida",
      "mobile_numbers": [],
      "email": "",
      "website": "",
      "GST": "OSAANCM5330P12T",
      "CIN": ""
    }
  ],
  "vendor_info": [
    {
      "vendor_name": "Awana Tour & Travels",
      "address": "Street No. 02, Gijhore, Near Kanchan Junga, Appartment, Sector-53, Noida. 2